# Analiza: Wykrywanie języka na podstawie częstotliwości słów w tekscie

Celem tej pracy jest ocena, czy na podstawie danych dotyczących częstotliwości występowania słów w arytkule na wybranym wiki (u nas bulbapedia) da się rozpoznać język tekstu. Eksperyment polega na porównaniu listy najczęsciej występujących słów na wiki z listą najczęściej występujących słów w jednym z trzech języków. Sprawdzimy dodatkowo jak na poprawność metody wpływa długość arytkułu.

In [34]:
# Początkowe importy najważniejszych modułów i bibliotek
import json
import re
import matplotlib.pyplot as plt
from wordfreq import top_n_list, zipf_frequency
from pathlib import Path
from scraper_logic import Scraper
from bs4 import BeautifulSoup
import requests

Wykorzystamy listy najczęsciej występujących słów z trzech języków: polskiego, angielskiego (język naszej wiki), oraz niemieckiego. Użyjemy do tego biblioteki wordfreq.

In [35]:
def get_language_words(lang, n=1000):
    words = top_n_list(lang, n)
    return {
        w : zipf_frequency(w, lang)
        for w in words 
    }
    
langs = ["en", "pl", "de"]

lang_freq = {lang: get_language_words(lang) for lang in langs}

Na potrzeby analizy stworzymy nowy uniwersalny scraper, który zadziała dla każdej strony, nie tylko bulbapedii.

In [36]:
# Nasza funkcja scrapująca zwraca obiekt bs, który potem będziemy edytować zależnie od struktury html
def get_article(url):
    # Tworzymy header, który pozwoli nam się dostać do stron używając headera z httpbin.org/cache
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:147.0) Gecko/20100101 Firefox/147.0'
    }

    try:
        source = requests.get(url, headers=headers)
    except requests.RequestException:
        return None

    if source.status_code != 200:
        print(f"Błąd: status_code={source.status_code} dla {url}")
        return None

    return BeautifulSoup(source.text, 'html.parser')


Jako dłuższy artykuł z wybranej wiki (bulbapedii) posłuży strona z opisami umiejętności pokemonów: [https://bulbapedia.bulbagarden.net/wiki/Ability]. Pobierzmy html tej strony używając naszej nowej funkcji.

In [37]:
lo_soup = get_article('https://bulbapedia.bulbagarden.net/wiki/Ability')

Zrobimy to samo dla krótkiego arytkuły ze strony [https://bulbapedia.bulbagarden.net/wiki/Bulbakaki], który pełen jest nazw własnych.

In [38]:
sh_soup = get_article('https://bulbapedia.bulbagarden.net/wiki/Bulbakaki')

Robimy to również dla artykułów w każdym z wybranych języków:

In [39]:
pl_soup = get_article('https://wolnelektury.pl/katalog/lektura/witkacy-o-czystej-formie.html')
en_soup = get_article('https://minecraft.fandom.com/wiki/Trading')
de_soup = get_article('https://wolnelektury.pl/katalog/lektura/heyse-die-schwarze-jakobe.html')

# Usuwamy polskie słowa z niemieckiego tekstu (bo pobieramy z polskiej strony)
for h2 in de_soup.find_all("h2"):
    h2.decompose()

Zdefiniujmy funkcję do liczenia słów zwracającą słownik:

In [40]:
def count_words(text) -> dict:
    word_dict = {}

    for word in text:
        word_dict[word] = word_dict.get(word, 0) + 1

    return word_dict


Teraz musimy w każdej z tych struktur znaleźć właściwą zawartość z artykułem, zamienić ją na tekst i policzyć słowa:

In [41]:
# Wycinanie właściwej zawartości (find zwraca typ tag)
sh_content = sh_soup.find('p')
lo_content = lo_soup.find('div', class_='mw-body-content')
pl_content = pl_soup.find('div', class_='main-text-body')
en_content = en_soup.find('div', id='content')
de_content = de_soup.find('div', class_='main-text-body')

contents = [sh_content, lo_content, pl_content, en_content, de_content]
dicts = []

for c in contents:
    # Zmieniamy typ na string
    text = c.get_text(' ', strip=True)
    # Parsujemy słowa (zwraca liste stringów)
    tokens = re.findall(r'[^\W\d_]+', text.lower())
    # Używamy naszej funckji do stworzenia słowników z licznikiem wystąpień
    res: dict[str, int] = count_words(tokens)

    dicts.append(res)

Upewnijmy się, że wszystko działa wypisując pare pierwszych słów:

In [42]:
# Dajemy pythonowi znać, że to obiekt typu dict
for d in dicts:
    print(list(d.items())[:100])

[('the', 3), ('bulbagarden', 2), ('oekaki', 2), ('known', 1), ('to', 3), ('its', 1), ('users', 2), ('as', 1), ('bulbakaki', 2), ('was', 2), ('an', 1), ('run', 1), ('by', 1), ('which', 1), ('used', 1), ('wacintaki', 1), ('poteto', 1), ('there', 1), ('is', 1), ('a', 1), ('deviantart', 1), ('group', 1), ('for', 1), ('of', 1), ('upload', 1), ('their', 1), ('completed', 1), ('artworks', 1), ('it', 1), ('closed', 1), ('down', 1), ('in', 1)]
[('the', 847), ('contents', 2), ('of', 237), ('this', 18), ('article', 9), ('have', 61), ('been', 6), ('suggested', 1), ('to', 280), ('be', 47), ('split', 2), ('into', 27), ('list', 8), ('abilities', 132), ('and', 241), ('by', 93), ('media', 1), ('appearance', 2), ('please', 2), ('discuss', 1), ('it', 168), ('on', 30), ('talk', 1), ('page', 1), ('for', 43), ('picture', 1), ('used', 110), ('in', 428), ('is', 108), ('unsatisfactory', 1), ('feel', 1), ('free', 1), ('replace', 1), ('so', 10), ('conforms', 1), ('bulbapedia', 3), ('conventions', 1), ('reason', 